# Getting started

In [ ]:
!pip install neo4j-driver

In [ ]:
# Installing neo4j server:
# $ sudo add-apt-repository "deb https://debian.neo4j.com stable 4.1"
# $ sudo apt-get install neo4j
# $ sudo systemctl enable neo4j.service
# $ sudo systemctl status neo4j.service

# Installing neo4j cliente
!apt-get install neo4j-client
# $ cypher-shell prompt

In [ ]:
# Doc:
# https://neo4j.com/developer/python/
# Neo4j as a service:
# https://browser.neo4j.io/
# https://console.neo4j.io/
from neo4j import GraphDatabase, basic_auth

In [ ]:
uri = "neo4j+s://888db127.databases.neo4j.io"
user = "neo4j"
password = "XXX"

driver = GraphDatabase.driver(uri, auth=basic_auth(user, password))
session = driver.session(database="neo4j")

In [ ]:
cypher_query = '''
MATCH (n) 
RETURN COUNT(n) AS count 
LIMIT $limit
'''

In [ ]:
results = session.read_transaction(
    lambda tx: tx.run(cypher_query,
                      limit=25).data())
for record in results:
    print(record['count'])

In [ ]:
movie_query = '''
MATCH (n:Movie)
RETURN n 
LIMIT $limit '''

In [ ]:
results = session.read_transaction(
    lambda tx: tx.run(movie_query,
                      limit=25).data())

In [ ]:
for record in results:
    print(record)

In [ ]:
follows_query = '''
MATCH p=()-[r:FOLLOWS]->() 
RETURN p 
LIMIT $limit'''


In [ ]:
results = session.read_transaction(
    lambda tx: tx.run(follows_query,
                      limit=25).data())

In [ ]:
for record in results:
    print(record['p'])

In [ ]:
driver.close()

In [ ]:
from neo4j import GraphDatabase

class HelloWorldExample:

    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self.driver.close()

    def print_greeting(self, message):
        with self.driver.session() as session:
            greeting = session.write_transaction(self._create_and_return_greeting, message)
            print(greeting)

    @staticmethod
    def _create_and_return_greeting(tx, message):
        result = tx.run("CREATE (a:Greeting) "
                        "SET a.message = $message "
                        "RETURN a.message + ', from node ' + id(a)", message=message)
        return result.single()[0]


if __name__ == "__main__":
    greeter = HelloWorldExample("bolt://localhost:7687", "neo4j", "password")
    greeter.print_greeting("hello, world")
    greeter.close()